In [150]:
##################################################################################
## Code to create the input digital models required for SOLWEIG: DEM, DSM, CDSM ##
##################################################################################

## Import the required libraries
import pdal
import numpy as np
import json
import rasterio
from rasterio.transform import from_origin
from rasterio import Affine
from rasterio.enums import Resampling
import os
from osgeo import gdal
import shutil


## Define all the functions to be used

#### Defining functions for merging and interpolating to fill gaps in DEM

def merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32):
    """
    Merges multiple raster files into a single raster using GDAL.

    Parameters:
        input_files (list): List of input raster file paths to merge.
        output_file (str): Path to the output merged raster file.
        nodata_value (float): NoData value for input and output rasters.
        data_type (GDAL DataType): Data type for the output raster (e.g., gdal.GDT_Float32).
        
    Returns:
        None
    """
    
    # Set the options for gdal.Warp
    warp_options = gdal.WarpOptions(
        format='GTiff',
        srcNodata=nodata_value,
        dstNodata=nodata_value,
        xRes=None,  # Keeps the original resolution
        yRes=None,
        creationOptions=['COMPRESS=LZW'],  # Compression option (optional)
        outputType=data_type
    )
    
    # Execute the merge operation
    gdal.Warp(output_file, input_files, options=warp_options)
    print(f"Raster files merged and saved to {output_file}")


### Filling in the gaps in the DSM
## using inverse weighting (idw)

def fill_nodata(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=0):
    """
    Fills NoData values in a DSM using GDAL's FillNodata function.

    Parameters:
        input_dsm_path (str): Path to the input DSM file.
        output_dsm_path (str): Path to save the output filled DSM file.
        max_distance (int): Maximum search distance (in pixels) to find values for interpolation.
        smoothing_iterations (int): Number of smoothing iterations to perform on the filled area.

    Returns:
        None
    """
    # Open the input DSM
    dsm_ds = gdal.Open(input_dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the input DSM.")
        return

    # Get raster band (assumes DSM has one band)
    dsm_band = dsm_ds.GetRasterBand(1)

    # Create output DSM with the same dimensions and georeferencing
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(
        output_dsm_path,
        dsm_ds.RasterXSize,
        dsm_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )

    # Copy georeference and projection from the input DSM
    output_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    output_ds.SetProjection(dsm_ds.GetProjection())

    # Read data and copy to the output band
    output_band = output_ds.GetRasterBand(1)
    dsm_data = dsm_band.ReadAsArray()

    # Set NoData value (assumes NoData value is already set in the input band)
    nodata_value = dsm_band.GetNoDataValue()
    output_band.SetNoDataValue(nodata_value)
    output_band.WriteArray(dsm_data)

    # Fill NoData values using GDAL's FillNodata function
    gdal.FillNodata(
        targetBand=output_band,
        maskBand=None,
        maxSearchDist=max_distance,
        smoothingIterations=smoothing_iterations
    )

    # Flush changes and close datasets
    output_band.FlushCache()
    output_ds = None
    dsm_ds = None
    print(f"Filled DSM saved to {output_dsm_path}")

## check if all of the gaps in the data have been filled

def check_nodata(dsm_path, nodata_value=-9999):
    """
    Checks if the DSM has any NoData values.

    Parameters:
        dsm_path (str): Path to the DSM file.
        nodata_value (float): NoData value to check for.
    
    Returns:
        bool: True if there are NoData values, False otherwise.
    """
    # Open the DEM
    dsm_ds = gdal.Open(dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the DEM.")
        return False

    # Get the raster band and read the data as an array
    dsm_band = dsm_ds.GetRasterBand(1)
    dsm_data = dsm_band.ReadAsArray()

    # Check if any NoData values are present
    has_nodata = np.any(dsm_data == nodata_value)
    dsm_ds = None  # Close the dataset
    return has_nodata

# keep filling the gaps until there are none left

def fill_gaps_until_complete(input_dsm, output_dsm, max_distance=100, smoothing_iterations=1):
    """
    Repeatedly fills NoData values in a DSM until no gaps remain.

    Parameters:
        input_dsm (str): Path to the input DSM file.
        output_dsm (str): Path to the output DSM file.
        max_distance (int): Maximum search distance for filling gaps.
        smoothing_iterations (int): Smoothing iterations during fill process.
    """
    
    iteration = 1
    fill_nodata(input_dsm, output_dsm, max_distance, smoothing_iterations)
    paths = [output_dsm]
    new_input_dsm_path = output_dsm
    # Continue filling gaps until no NoData values remain
    while check_nodata(new_input_dsm_path, nodata_value=-9999):
        print(f"Iteration {iteration}: Filling NoData values...")
        new_input_dsm_path = paths[iteration - 1]
        new_output_dsm_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
        paths.append(new_output_dsm_path)
        fill_nodata(new_input_dsm_path, new_output_dsm_path, max_distance, smoothing_iterations)
        iteration += 1
    
    print("No more NoData values remain. Filling process completed.")

## defining function for converting the dms from feet to meters 

def convert_dsm_feet_to_meters(input_dsm_path, output_dsm_path):
    """
    Converts a DSM raster from feet to meters.

    Parameters:
        input_dsm_path (str): Path to the input DSM file in feet.
        output_dsm_path (str): Path to the output DSM file in meters.

    Returns:
        None
    """
    # Open the input DSM
    dsm_ds = gdal.Open(input_dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the input DSM.")
        return

    # Get the raster band and NoData value
    dsm_band = dsm_ds.GetRasterBand(1)
    nodata_value = dsm_band.GetNoDataValue()
    dsm_data = dsm_band.ReadAsArray()

    # Conversion factor from feet to meters
    feet_to_meters = 0.3048

    # Convert DSM values from feet to meters
    dsm_data_meters = np.where(dsm_data != nodata_value, dsm_data * feet_to_meters, nodata_value)

    # Create the output DSM with the same dimensions and georeferencing
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(
        output_dsm_path,
        dsm_ds.RasterXSize,
        dsm_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )

    # Copy geotransform and projection from the input DSM
    output_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    output_ds.SetProjection(dsm_ds.GetProjection())

    # Write the converted data to the output file
    output_band = output_ds.GetRasterBand(1)
    output_band.SetNoDataValue(nodata_value)
    output_band.WriteArray(dsm_data_meters)

    # Flush changes and close datasets
    output_band.FlushCache()
    output_ds = None
    dsm_ds = None
    print(f"DSM successfully converted to meters and saved to {output_dsm_path}")





In [151]:
# get the laz files in the central area of Durham
file_path = "D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles"  
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(file_path):
    if file_name.endswith(".laz"):
        file_name_list.append(file_name)


In [152]:
file_name_list

['Job1021395_35078_95_88.laz',
 'Job1021395_35078_95_91.laz',
 'Job1021395_35078_95_94.laz',
 'Job1021395_35078_98_88.laz',
 'Job1021395_35078_98_91.laz',
 'Job1021395_35078_98_94.laz',
 'Job1021395_36078_01_88.laz',
 'Job1021395_36078_01_91.laz',
 'Job1021395_36078_01_94.laz']

In [153]:
file_path

'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles'

In [155]:

# for the chosen file, extract required info to build the dms
# Classify either ground, roads, trees or buildings
# Ground = 2, Roads = 13, Trees = 5, Buildings = 6

classifications_folder = 'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/'

classify_list = ['ground','roads','trees','buildings']

for laz_filename in file_name_list:
    print('Starting processing for', laz_filename)
    for classify in classify_list:
        if classify == 'ground':
            class_num = 2
        if classify == 'roads':
            class_num = 13
        if classify == 'trees':
            class_num = 5
        if classify == 'buildings':
            class_num = 6

        print('Classification:',classify)
        # Input and output files
        input_las = file_path+'\\'+laz_filename
        output_dem = file_path+'\\Classifications\\'+classify+'_'+laz_filename[:-3]+'tif'

        fileinfo = laz_filename[:-4]

        # PDAL Pipeline JSON for processing
        pipeline = {
                "pipeline": [
                {
                    "type": "readers.las",
                    "filename": input_las
                },
                {
                    "type": "filters.range",
                    "limits": "Classification[{}:{}]".format(class_num,class_num)  # remember to set the classification type above!
                },
                {
                    "type": "filters.ferry",
                    "dimensions": "Z=HeightAboveGround"  # Optional: Rename Z to HeightAboveGround
                },
                {
                    "type": "writers.gdal",
                    "filename": output_dem,
                    "resolution": 1.0,  # Define resolution 1-meter grid
                    "output_type": "idw",  # value for the pixel idw is shepards inverse distance weighting for points in radius, mean, min, max as expected
                    "data_type": "float32",
                    "nodata": -9999
                }
            ]
        }

        # Convert the pipeline to a JSON string
        pipeline_json = json.dumps(pipeline)

        # Execute the pipeline
        p = pdal.Pipeline(pipeline_json)

        # Execute the pipeline
        try:
            p.execute()
            print("DM generation successful.")
        except RuntimeError as e:
            print("PDAL pipeline execution failed:", e)

# Load the DEM using Rasterio to check metadata (optional)
#with rasterio.open(output_dem) as dem:
#    print(f"DEM Metadata:\n {dem.meta}")
#    print("DEM preview (5x5):")
#    data = dem.read(1)  # Read the first band
#    print(data[:5, :5])

    ##################################################################
    ##################### CREATING THE DEM ###########################
    ##################################################################

    ### merge the ground and road layers
    ground_filename = 'ground_{}.tif'.format(fileinfo)
    road_filename = 'roads_{}.tif'.format(fileinfo)
    ground_filepath = os.path.join(classifications_folder, ground_filename)
    road_filepath = os.path.join(classifications_folder, road_filename)

    input_files = [ground_filepath, road_filepath]

    output_file = os.path.join(classifications_folder, 'merged_output.tif')

    merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32)

    ## fill in the missing data in the merged ground+road layer
    input_dsm_path = output_file
    output_dsm_path =  os.path.join(classifications_folder, 'DEM_python.tif')

    #fill_nodata(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=1)
    #check_nodata(output_dsm_path, nodata_value=-9999)
    fill_gaps_until_complete(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=1)

    # copy the filled DEM into folder for DEMs
    final_dem_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
    folder_for_final_dems = os.path.join(classifications_folder,'DEMs')
    DEM_filename = 'DEM_{}.tif'.format(fileinfo)
    DEM_filepath = os.path.join(folder_for_final_dems, DEM_filename)
    shutil.copy(final_dem_path, DEM_filepath)

    ##################################################################
    ##################### CREATING THE DSM ###########################
    ##################################################################

    ### merge the DEM and buildings
    buildings_filename = 'buildings_{}.tif'.format(fileinfo)
    buildings_filepath = os.path.join(classifications_folder, buildings_filename)

    input_files = [DEM_filepath, buildings_filepath]

    folder_for_final_dsms = os.path.join(classifications_folder,'DSMs')
    DSM_filename = 'DSM_{}.tif'.format(fileinfo)

    DSM_filepath = os.path.join(folder_for_final_dsms, DSM_filename)

    merge_rasters(input_files, DSM_filepath, nodata_value=-9999, data_type=gdal.GDT_Float32)

    ##################################################################
    ##################### CREATING THE CDSM ###########################
    ##################################################################

    ### merge the DEM and tree canopy
    trees_filename = 'trees_{}.tif'.format(fileinfo)
    trees_filepath = os.path.join(classifications_folder, trees_filename)


    ### merge the DEM and trees
    input_files = [ DEM_filepath, trees_filepath]

    folder_for_trees_DEM = os.path.join(classifications_folder, 'trees_plus_DEM')
    trees_DEM_filename = 'trees_DEM_{}.tif'.format(fileinfo)
    trees_DEM_filepath = os.path.join(folder_for_trees_DEM, trees_DEM_filename)

    merge_rasters(input_files, trees_DEM_filepath, nodata_value=-9999, data_type=gdal.GDT_Float32)

    # subtract the DEM from the trees+DEM raster
    # add in condition that values over 150 ft are filtered out, as these are outliers 
    # path to save output to
    CDSM_filename = 'CDSM_{}.tif'.format(fileinfo)
    folder_for_final_cdsms = os.path.join(classifications_folder,'CDSMs')
    CDSM_filepath = os.path.join(folder_for_final_cdsms, CDSM_filename)

    # Define the maximum threshold value for the tree heights (NOTE: this is in feet not meters!!)
    threshold_value = 150

    # Open the raster files
    with rasterio.open(trees_DEM_filepath) as src1, rasterio.open(DEM_filepath) as src2:
        # Check if both rasters have the same shape
        if src1.width != src2.width or src1.height != src2.height:
            raise ValueError("Rasters must have the same dimensions.")
    
        # Read the first band from each raster
        tree_DEM_raster = src1.read(1)
        DEM_raster = src2.read(1)

        # Perform raster subtraction (handling NoData values)
        nodata = src1.nodata   # get the values for no data, should be -9999
        result = np.where(
            (tree_DEM_raster == nodata) | (DEM_raster == nodata), nodata, tree_DEM_raster - DEM_raster
        )
    
        # Apply threshold filtering: Set values greater than 150 to NoData
        result = np.where(result > threshold_value, nodata, result)
    
        # Define the metadata for the output raster
        out_meta = src1.meta.copy()
        out_meta.update({
            "dtype": "float32",
            "nodata": nodata
        })

        # Write the result to a new raster file
        with rasterio.open(CDSM_filepath, "w", **out_meta) as dst:
            dst.write(result, 1)

    print(f"Raster subtraction completed. Output saved as {CDSM_filepath}.")

Classification: ground
DM generation successful.
Classification: roads
DM generation successful.
Classification: trees
DM generation successful.
Classification: buildings
DM generation successful.
Raster files merged and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/merged_output.tif
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python.tif
Iteration 1: Filling NoData values...
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python_iteration1.tif
Iteration 2: Filling NoData values...
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python_iteration2.tif
No more NoData values remain. Filling process completed.
Raster files merged and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs\DSM_Job1021395_35078_98_94.tif
Raster files merged and saved to D:/laz_format_files_Durha

In [161]:
## convert the heights of digital models into meters (currently are in feet)

# DEMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

# DSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DSM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

#CDSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/CDSMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/CDSM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)


DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_98_94.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DSM\DSM_Job1021395_35078_98_94.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/CDSM\CDSM_Job1021395_35078_98_94.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/CDSM\DSM_Job1021395_35078_98_94.tif


In [ ]:
def merge_tifs(input_folder, merged_output_path):
    """
    Merges multiple GeoTIFF files into a single raster.

    Parameters:
        input_folder (str): Path to the folder containing the input GeoTIFF files.
        merged_output_path (str): Path to save the merged output GeoTIFF.

    Returns:
        str: The path to the merged GeoTIFF file.
    """
    # List all TIFF files in the input folder
    tif_files = glob.glob(os.path.join(input_folder, "*.tif"))

    if not tif_files:
        print("No TIFF files found in the input folder.")
        return None

    # Use gdal.Warp to merge the TIFF files
    print("Merging input TIFF files...")
    gdal.Warp(merged_output_path, tif_files, format="GTiff", options=["COMPRESS=LZW"])
    print(f"Merged DEM saved to {merged_output_path}")
    return merged_output_path

def split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=2000, buffer_size=150):
    """
    Splits a DEM into smaller tiles of 2000x2000 meters with a 150-meter buffer.

    Parameters:
        input_dem_path (str): Path to the input DEM file.
        output_folder (str): Folder to save the output tiles.
        tile_size (int): Size of each tile in meters (without buffer).
        buffer_size (int): Buffer size to add to each tile in meters.

    Returns:
        None
    """
    
    # Open the input DEM
    dem_ds = gdal.Open(input_dem_path)
    if dem_ds is None:
        print("Failed to open the input DEM.")
        return

    # Get georeferencing information
    geotransform = dem_ds.GetGeoTransform()
    min_x = geotransform[0]
    max_y = geotransform[3]
    pixel_size_x = geotransform[1]
    pixel_size_y = abs(geotransform[5])
    cols = dem_ds.RasterXSize
    rows = dem_ds.RasterYSize

    # Calculate the spatial extent of the DEM
    max_x = min_x + cols * pixel_size_x
    min_y = max_y - rows * pixel_size_y

    # Calculate the size of tiles with buffer
    full_tile_size = tile_size + 2 * buffer_size

    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through the grid, creating tiles
    x_tiles = math.ceil((max_x - min_x) / tile_size)
    y_tiles = math.ceil((max_y - min_y) / tile_size)

    for i in range(x_tiles):
        for j in range(y_tiles):
            # Calculate the boundaries of the tile with buffer
            tile_min_x = min_x + i * tile_size - buffer_size
            tile_max_x = min_x + (i + 1) * tile_size + buffer_size
            tile_max_y = max_y - j * tile_size + buffer_size
            tile_min_y = max_y - (j + 1) * tile_size - buffer_size

            # Clip to DEM extent
            tile_min_x = max(tile_min_x, min_x)
            tile_max_x = min(tile_max_x, max_x)
            tile_min_y = max(tile_min_y, min_y)
            tile_max_y = min(tile_max_y, max_y)

            # Calculate pixel coordinates of the tile
            offset_x = int((tile_min_x - min_x) / pixel_size_x)
            offset_y = int((max_y - tile_max_y) / pixel_size_y)
            width = int((tile_max_x - tile_min_x) / pixel_size_x)
            height = int((tile_max_y - tile_min_y) / pixel_size_y)

            # Generate output file path
            tile_filename = f"tile_{i}_{j}.tif"
            output_tile_path = os.path.join(output_folder, tile_filename)

            # Extract the tile using gdal.Translate
            gdal.Translate(
                output_tile_path,
                dem_ds,
                srcWin=[offset_x, offset_y, width, height],
                outputType=gdal.GDT_Float32,
                creationOptions=["COMPRESS=LZW"]
            )

            print(f"Created tile: {output_tile_path}")

    # Close the DEM dataset
    dem_ds = None
    print("DEM splitting completed.")

# Main function to merge and split DEMs
def merge_and_split_dems(input_folder, merged_output_path, output_folder, tile_size=2000, buffer_size=150):
    """
    Merges TIFF files in the input folder and splits the merged DEM into smaller tiles.

    Parameters:
        input_folder (str): Path to the folder containing input GeoTIFF files.
        merged_output_path (str): Path to save the merged GeoTIFF file.
        output_folder (str): Folder to save the output tiles.
        tile_size (int): Size of each tile in meters (without buffer).
        buffer_size (int): Buffer size to add to each tile in meters.
    
    Returns:
        None
    """
    # Merge TIFF files
    merged_dem_path = merge_tifs(input_folder, merged_output_path)
    if merged_dem_path:
        # Split merged DEM into tiles
        split_dem_into_tiles_with_buffer(merged_dem_path, output_folder, tile_size, buffer_size)

# Example usage
input_folder = "D:/example/source_folder"
merged_output_path = "D:/example/merged_dem.tif"
output_folder = "D:/example/tiles"
merge_and_split_dems(input_folder, merged_output_path, output_folder)

In [ ]:
input_files

In [157]:
def get_raster_size(raster_path):
    with rasterio.open(raster_path) as src:
        width = src.width
        height = src.height
        return width, height

raster_file = "D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs/DSM_Job1021395_35078_98_94.tif"
width, height = get_raster_size(raster_file)

print("Raster width:", width)
print("Raster height:", height)

Raster width: 8882
Raster height: 10927


In [158]:
with rasterio.open(raster_file) as src:
    # Get basic metadata
    print("Width:", src.width)
    print("Height:", src.height)
    print("Number of bands:", src.count)
    print("Coordinate Reference System (CRS):", src.crs)
    print("Bounding box:", src.bounds)
    print("Data type:", src.dtypes)
    print("Driver:", src.driver)

Width: 8882
Height: 10927
Number of bands: 1
Coordinate Reference System (CRS): COMPD_CS["NAD83(2011) / North Carolina (ftUS); NAVD88 height (ftUS)",PROJCS["NAD83(2011) / North Carolina (ftUS)",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1116"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6318"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",33.75],PARAMETER["central_meridian",-79],PARAMETER["standard_parallel_1",36.1666666666667],PARAMETER["standard_parallel_2",34.3333333333333],PARAMETER["false_easting",2000000],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","6543"]],VERT_CS["NAVD88 height (ftUS)",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORI

In [89]:
### merge the DSM and trees

input_files = [
    'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs\\DSM_Job1021395_35078_98_94.tif',
    'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_Job1021395_35078_98_94.tif'
]

folder_for_trees_DEM = 'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_plus_DEM'
new_filename = 'trees_DSM_{}.tif'.format(fileinfo)

output_file = os.path.join(folder_for_trees_DEM, new_filename)

merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32)


Raster files merged and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_plus_DEM\trees_DSM_Job1021395_35078_98_94.tif
